In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# import os

# # Define the path to the main dataset directory
# dataset_dir = "/kaggle/input/vehicle-images-gti"

# # Dictionary to hold the label names and their respective file paths
# dataset_structure = {}

# # Walk through the dataset directory
# for root, dirs, files in os.walk(dataset_dir):
#     # Only consider subdirectories directly under the main dataset directory
#     if root == dataset_dir:
#         for folder_name in dirs:
#             folder_path = os.path.join(root, folder_name)
#             dataset_structure[folder_name] = folder_path

# # Print the dataset structure
# print("Dataset structure:")
# for label, path in dataset_structure.items():
#     print(f"Folder: {label}, Path: {path}")

In [ ]:
# import os
# import shutil

# # Define source directories
# dir1 = "/kaggle/input/vehicle-images-gti/VehicleImage/non-vehicles"
# dir2 = "/kaggle/input/vehicle-images-gti/VehicleImage/vehicles"

# # Define destination directory within Kaggle's working directory
# destination_dir = "/kaggle/working/combined_images"

# # Create the destination directory if it doesn't exist
# os.makedirs(destination_dir, exist_ok=True)

# # List of label folders in each directory
# label_folders = ["Far", "Left", "MiddleClose", "Right"]

# # Copy images from each label folder in both directories to the destination directory
# for label in label_folders:
#     # Paths to the label folders in each source directory
#     dir1_label_path = os.path.join(dir1, label)
#     dir2_label_path = os.path.join(dir2, label)
    
#     # Create the corresponding label folder in the destination directory
#     dest_label_path = os.path.join(destination_dir, label)
#     os.makedirs(dest_label_path, exist_ok=True)
    
#     # Copy images from dir1 label folder to destination label folder
#     for filename in os.listdir(dir1_label_path):
#         file_path = os.path.join(dir1_label_path, filename)
#         if os.path.isfile(file_path):
#             shutil.copy(file_path, dest_label_path)
    
#     # Copy images from dir2 label folder to destination label folder
#     for filename in os.listdir(dir2_label_path):
#         file_path = os.path.join(dir2_label_path, filename)
#         if os.path.isfile(file_path):
#             shutil.copy(file_path, dest_label_path)

# print("All images have been successfully copied to the destination directory.")


In [ ]:
# import shutil
# import os

# # Define the path to the working directory
# destination_dir = "/kaggle/working/combined_images"

# # Check if the directory exists, then delete it
# if os.path.exists(destination_dir):
#     shutil.rmtree(destination_dir)
#     print("The working directory has been deleted.")
# else:
#     print("The specified directory does not exist.")


In [ ]:
import os
import shutil

# Define source directories
non_vehicle_dir = "/kaggle/input/vehicle-images-gti/VehicleImage/non-vehicles"
vehicle_dir = "/kaggle/input/vehicle-images-gti/VehicleImage/vehicles"

# Define destination directory within Kaggle's working directory
destination_dir = "/kaggle/working/combined_images"
os.makedirs(destination_dir, exist_ok=True)

# List of label folders within each category
label_folders = ["Far", "Left", "MiddleClose", "Right"]

# Dictionary to map original folders to new folder names
category_map = {
    "vehicle": vehicle_dir,
    "non_vehicle": non_vehicle_dir
}

# Copy images from each label folder in both categories to the destination directory
for category, base_dir in category_map.items():
    for label in label_folders:
        # Paths to the label folders in each source directory
        source_label_path = os.path.join(base_dir, label)
        
        # Create the new folder in the destination directory (e.g., vehicle_far, non_vehicle_left)
        new_folder_name = f"{category.lower()}_{label.lower()}"
        dest_label_path = os.path.join(destination_dir, new_folder_name)
        os.makedirs(dest_label_path, exist_ok=True)
        
        # Copy images from the source label folder to the new destination label folder
        if os.path.exists(source_label_path):
            for filename in os.listdir(source_label_path):
                file_path = os.path.join(source_label_path, filename)
                if os.path.isfile(file_path):
                    shutil.copy(file_path, dest_label_path)

# Optional: Clean up any unintended folders in the destination directory
for folder in os.listdir(destination_dir):
    folder_path = os.path.join(destination_dir, folder)
    if os.path.isdir(folder_path) and folder not in [f"{cat.lower()}_{label.lower()}" for cat in category_map.keys() for label in label_folders]:
        shutil.rmtree(folder_path)

print("Images have been successfully reorganized into 8 folders.")


In [ ]:
import os

# Define the path to the destination directory
destination_dir = "/kaggle/working/combined_images"

# List the folders in the destination directory
class_folders = [folder for folder in os.listdir(destination_dir) if os.path.isdir(os.path.join(destination_dir, folder))]

# Count the number of classes
num_classes = len(class_folders)

print("Number of classes:", num_classes)
print("Classes:", class_folders)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

# Paths to training and validation datasets
train_dir = "/kaggle/working/combined_images"

# Image data generator with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,  # Use 20% of data for validation
)

# Load training and validation sets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)



In [ ]:
# Load the VGG16 model with pre-trained ImageNet weights
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_base.trainable = False  # Freeze the base model

# Build the model
model = models.Sequential([
    vgg16_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax')  # 8 classes for your categories
])




In [ ]:
# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)



In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
# Create a figure and two subplots for accuracy and loss
plt.figure(figsize=(10, 5))

# Plot Training and Validation Accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')

# Plot Training and Validation Loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')

# Add title and labels
plt.title('Training and Validation Accuracy & Loss')
plt.xlabel('Epoch')
plt.ylabel('Accuracy / Loss')

# Add a legend to differentiate between lines
plt.legend()

# Display the plot
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, mean_squared_error
from tensorflow.keras.utils import to_categorical

# Make predictions on the validation set
y_pred_probs = model.predict(validation_generator)
y_pred = np.argmax(y_pred_probs, axis=1)  # Get predicted classes

# Get true labels from validation generator
y_true = validation_generator.classes

# Convert to categorical if needed for specific metrics
y_true_categorical = to_categorical(y_true)
y_pred_categorical = to_categorical(y_pred)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f"RMSE: {rmse:.2f}")

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"Accuracy (ACC): {accuracy * 100:.2f}%")

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

# Calculate TP, FP, TN, FN, Specificity, and Sensitivity
TP = np.diag(conf_matrix)  # True positives per class
FP = np.sum(conf_matrix, axis=0) - TP  # False positives per class
FN = np.sum(conf_matrix, axis=1) - TP  # False negatives per class
TN = np.sum(conf_matrix) - (FP + FN + TP)  # True negatives per class

# Average values for binary specificity and sensitivity
specificity = TN / (TN + FP)
sensitivity = TP / (TP + FN)

print(f"True Positives (TP): {TP}")
print(f"False Positives (FP): {FP}")
print(f"True Negatives (TN): {TN}")
print(f"False Negatives (FN): {FN}")
print(f"Specificity (SPE): {specificity}")
print(f"Sensitivity (SENS): {sensitivity}")

# Calculate AUC
try:
    auc = roc_auc_score(y_true_categorical, y_pred_probs, multi_class="ovr")
    print(f"Area Under Curve (AUC): {auc:.2f}")
except ValueError:
    print("AUC could not be calculated (might need binary classification or one-vs-rest strategy).")


In [ ]:
import matplotlib.pyplot as plt

# Plot RMSE, Accuracy, and AUC
plt.figure(figsize=(10, 5))
metrics = ['RMSE', 'Accuracy', 'AUC']
values = [rmse, accuracy, auc]

plt.subplot(1, 2, 1)
plt.bar(metrics, values, color=['blue', 'green', 'purple'])
plt.title("Overall Metrics")
plt.ylabel("Score")
plt.ylim(0, 1.1)  # Adjust based on range of metrics

# Plot TP, FP, TN, FN, Specificity, and Sensitivity per class
classes = np.arange(len(TP))  # Create an index for each class

plt.subplot(1, 2, 2)
width = 0.15  # Bar width
plt.bar(classes - width * 2, TP, width, label='TP', color='blue')
plt.bar(classes - width, FP, width, label='FP', color='orange')
plt.bar(classes, TN, width, label='TN', color='green')
plt.bar(classes + width, FN, width, label='FN', color='red')
plt.bar(classes + width * 2, specificity, width, label='Specificity', color='purple')
plt.bar(classes + width * 3, sensitivity, width, label='Sensitivity', color='pink')

plt.xlabel("Classes")
plt.ylabel("Count / Rate")
plt.title("Class-wise Metrics")
plt.legend()
plt.xticks(classes, [f'Class {i}' for i in classes])  # Label each class

plt.tight_layout()
plt.show()
